## data

In [16]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv("Clean_Dataset.csv")
display(df.head())

,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


## add departure and arrival date

In [18]:
df.drop("arrival_time",axis=1,inplace=True)

In [19]:
df.drop("departure_time",axis=1,inplace=True)

In [20]:
df.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

In [21]:
df.head()

,id,airline,flight,source_city,stops,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,zero,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,zero,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,zero,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,zero,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,zero,Mumbai,Economy,2.33,1,5955


In [22]:
import random
from datetime import datetime, timedelta


dates = []

def random_date(start_date, end_date):
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    random_seconds = random.randint(0, 24*3600 - 1)  # Gün içindeki rastgele saniye sayısı
    return start_date + timedelta(days=random_days, seconds=random_seconds)

start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 7, 20)

for _ in range(300153):
    random_date_obj = random_date(start_date, end_date)
    random_date_str = random_date_obj.strftime("%Y-%m-%d %H:%M:%S")
    dates.append(random_date_str)

# Mevcut DataFrame'e yeni bir kolon olarak tarih ve saat ekleyelim
df['departure_date'] = dates
display(df.head())

,id,airline,flight,source_city,stops,destination_city,class,duration,days_left,price,departure_date
0,0,SpiceJet,SG-8709,Delhi,zero,Mumbai,Economy,2.17,1,5953,2024-03-03 10:58:10
1,1,SpiceJet,SG-8157,Delhi,zero,Mumbai,Economy,2.33,1,5953,2024-06-10 20:19:31
2,2,AirAsia,I5-764,Delhi,zero,Mumbai,Economy,2.17,1,5956,2023-01-21 01:48:12
3,3,Vistara,UK-995,Delhi,zero,Mumbai,Economy,2.25,1,5955,2023-02-02 13:41:57
4,4,Vistara,UK-963,Delhi,zero,Mumbai,Economy,2.33,1,5955,2023-01-29 12:22:38


In [23]:
df['departure_date'] = pd.to_datetime(df['departure_date'])
df['arrival_date'] = df.apply(lambda row: row['departure_date'] + timedelta(hours=row['duration']), axis=1)

display(df.head())

,id,airline,flight,source_city,stops,destination_city,class,duration,days_left,price,departure_date,arrival_date
0,0,SpiceJet,SG-8709,Delhi,zero,Mumbai,Economy,2.17,1,5953,2024-03-03 10:58:10,2024-03-03 13:08:22
1,1,SpiceJet,SG-8157,Delhi,zero,Mumbai,Economy,2.33,1,5953,2024-06-10 20:19:31,2024-06-10 22:39:19
2,2,AirAsia,I5-764,Delhi,zero,Mumbai,Economy,2.17,1,5956,2023-01-21 01:48:12,2023-01-21 03:58:24
3,3,Vistara,UK-995,Delhi,zero,Mumbai,Economy,2.25,1,5955,2023-02-02 13:41:57,2023-02-02 15:56:57
4,4,Vistara,UK-963,Delhi,zero,Mumbai,Economy,2.33,1,5955,2023-01-29 12:22:38,2023-01-29 14:42:26


## split data

In [24]:
flight=df[["id","flight","source_city","stops","destination_city","duration","days_left","price","departure_date","arrival_date"]]

In [25]:
airline=df[["id","airline"]]

In [26]:
classes=df[["id","class"]]

## connecting to database

In [12]:
pip install pandas sqlalchemy psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [27]:
import pandas as pd
from sqlalchemy import create_engine

username = 'postgres'
password = '1009'
host = 'postgres'
port = '5432'
database_name = 'postgres'

# config.py
conn_url = f'postgresql://{username}:{password}@{host}:{port}/{database_name}'

# postgres_dao.py
engine = create_engine(conn_url)


## create tables

In [13]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [28]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, DateTime

In [14]:
# DataFrame'i PostgreSQL'e kaydetme
table_name = 'airline'

# SQLAlchemy MetaData objesi oluşturma
metadata = MetaData()

# users tablosu için SQLAlchemy Table objesi oluşturma
airline_table = Table(
    table_name, metadata,
    Column('id', Integer, primary_key=True),
    Column('airline', String, nullable=False)
)

# Tabloyu PostgreSQL veritabanına yükleme (oluşturma)
metadata.create_all(engine)

print("airline tablosu başarıyla oluşturuldu.")

airline tablosu başarıyla oluşturuldu.


In [15]:
# DataFrame'i PostgreSQL'e kaydetme
table_name = 'classes'

# SQLAlchemy MetaData objesi oluşturma
metadata = MetaData()

# users tablosu için SQLAlchemy Table objesi oluşturma
classes_table = Table(
    table_name, metadata,
    Column('id', Integer, primary_key=True),
    Column('class', String, nullable=False)
)

# Tabloyu PostgreSQL veritabanına yükleme (oluşturma)
metadata.create_all(engine)

print("classes tablosu başarıyla oluşturuldu.")

classes tablosu başarıyla oluşturuldu.


In [16]:
# DataFrame'i PostgreSQL'e kaydetme
table_name = 'flight'

# SQLAlchemy MetaData objesi oluşturma
metadata = MetaData()

# users tablosu için SQLAlchemy Table objesi oluşturma
flight_table = Table(
    table_name, metadata,
    Column('id', Integer, primary_key=True),
    Column('flight', String, nullable=False),
    Column('source_city', String, nullable=False),
    Column('stops', String, nullable=False),
    Column('destination_city', String, nullable=False),
    Column('duration', Float),
    Column('days_left', Integer, primary_key=True),
    Column('price', Integer, primary_key=True),
    Column('departure_date', DateTime),
    Column('arrival_date', DateTime)
)

# Tabloyu PostgreSQL veritabanına yükleme (oluşturma)
metadata.create_all(engine)

print("flight tablosu başarıyla oluşturuldu.")

flight tablosu başarıyla oluşturuldu.


## adding data

In [45]:
import psycopg2
# PostgreSQL bağlantı bilgilerini yükleme
config = {'host': 'postgres',
          'database': 'postgres',
          'user': 'postgres',
          'password': '1009',
           'port': '5432'}

In [46]:
# PostgreSQL'e bağlanma

# data/dao/postgres_dao.py
conn = psycopg2.connect(**config)
cur = conn.cursor()

In [31]:
# DataFrame'deki verileri PostgreSQL'e ekleme

# DataFrame'den tuple listesi oluşturma
airline_list = [(row['id'], row['airline']) for index, row in airline.iterrows()]
class_list = [(row['id'], row['class']) for index, row in classes.iterrows()]
flight_list = [(row['id'], row['flight'], row['source_city'], row['stops'], row['destination_city'], row['duration'], row['days_left'], row['price'], row['departure_date'], row['arrival_date']) for index, row in flight.iterrows()]


# data/dao/sql_query.py
# SQL INSERT sorgusu
sql_1= "INSERT INTO airline(id, airline) VALUES(%s, %s)"
sql_2 = "INSERT INTO classes(id, class) VALUES(%s, %s)"
sql_3= "INSERT INTO flight(id, flight, source_city,stops,destination_city,duration,days_left,price,departure_date,arrival_date) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

In [20]:
# data/dao/postgres_dao.py/SaveIntoPostgres() save_multiple_entities_into_postgres()
cur.executemany(sql_1, airline_list)
cur.executemany(sql_2, class_list)
cur.executemany(sql_3, flight_list)

In [21]:
conn.commit()

## merging tables

In [32]:
end_date

datetime.datetime(2024, 7, 20, 0, 0)

In [33]:
from datetime import timedelta ,datetime

In [34]:
# Son iki haftanın başlangıç ve bitiş tarihlerini hesapla
end_date = datetime(2024, 7, 20)
start_date = end_date - timedelta(days=1)

In [35]:
 # SQL sorgusu oluşturma (INNER JOIN ile tabloları birleştirme)
# data/dao/sql_query.py
sql = f"""
    SELECT flight.id, flight.flight, flight.source_city, flight.destination_city, flight.price , flight.departure_date ,flight.arrival_date, classes.class, airline.airline
    FROM flight
    INNER JOIN classes ON flight.id = classes.id
    INNER JOIN airline ON flight.id = airline.id
    WHERE flight.departure_date >= %s AND flight.departure_date <= %s;
"""

# SQL sorgusunu çalıştırma
# data/dao/postgres_dao.py/SaveIntoPostgres() save_entity_into_postgres()
#def save_entity_into_postgres(query, data):
    #cur.execute(query, data)

# save.save_entity_into_postgres(sql, (start_date, end_date))

In [36]:
cur.execute('select * from  flight  LIMIT 5;')

In [37]:
cur.execute(sql, (start_date, end_date))

In [38]:
columns = ['id', 'flight', 'source_city', 'destination_city','price','departure_date','arrival_date','class','airline']
rows = cur.fetchall()
df_merged = pd.DataFrame(rows, columns=columns)

In [39]:
rows = cur.fetchall()

In [40]:
columns = ['id', 'flight', 'source_city', 'destination_city','price','departure_date','arrival_date','class','airline']

In [41]:
rows = cur.fetchall()

In [42]:
df_merged

,id,flight,source_city,destination_city,price,departure_date,arrival_date,class,airline
0,32,UK-813,Delhi,Mumbai,12150,2024-07-19 02:23:28,2024-07-19 17:03:40,Economy,Vistara
1,41,AI-502,Delhi,Mumbai,12150,2024-07-19 18:14:51,2024-07-20 13:19:39,Economy,Air_India
2,382,UK-819,Delhi,Mumbai,21233,2024-07-19 00:08:16,2024-07-19 07:43:04,Economy,Vistara
3,1327,G8-719,Delhi,Mumbai,5954,2024-07-19 12:10:12,2024-07-20 02:25:12,Economy,GO_FIRST
4,1588,UK-833,Delhi,Mumbai,13725,2024-07-19 13:03:16,2024-07-19 20:23:04,Economy,Vistara
...,...,...,...,...,...,...,...,...,...
522,293178,AI-539,Chennai,Mumbai,61649,2024-07-19 03:55:01,2024-07-19 23:44:49,Business,Air_India
523,294473,UK-824,Chennai,Bangalore,44144,2024-07-19 20:38:58,2024-07-20 19:23:58,Business,Vistara
524,296008,UK-834,Chennai,Kolkata,55377,2024-07-19 11:24:55,2024-07-20 12:20:07,Business,Vistara
525,298724,UK-838,Chennai,Hyderabad,51457,2024-07-19 15:53:41,2024-07-20 07:23:41,Business,Vistara


In [91]:
cur.close()
conn.close()

In [ ]:
# vs code
# python extensions 

## crewai

In [100]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import time
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool

def show_time(message, t1):
    t2 = time.time()
    print(f"{message}: {round(t2-t1,4)} saniye")
    return t2

# OpenAI ortam değişkenlerini tanımlayın
os.environ["OPENAI_API_BASE"] = 'https://api.groq.com/openai/v1'
os.environ["OPENAI_MODEL_NAME"] = 'llama3-70b-8192'
os.environ["OPENAI_API_KEY"] = "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"


# Ajanları tanımlayın
analyst = Agent(
    role='Veri Analisti',
    goal='Verilen veri kümesi hakkında detaylı analiz yap ve veri ile ilgili bilgileri araştır.',
    backstory="""
    Sen uzman bir araştırmacısın. 
    Araştırma yaparken İngilizce kullanıyorsun.
    Teknik konulara hakimsin. 
    Belirli bir veri kümesi hakkında derinlemesine bilgi toplaman gerekiyor.
    """,
    verbose=True,
    allow_delegation=False
)

writer = Agent(
    role='Raporlayıcı',
    goal='Araştırmacının sağladığı bilgileri kullanarak en yüksek geliri elde edebilmek için uçak fiyatları nasıl belirlenmelidir hangi parametreler daha etkilidir yorumlamalar yap.',
    backstory="""
    Sen uzman bir fiyat raporlayıcısısın. 
    Teknik konulara hakimsin. 
    Teknik ve bilgilendirici içerik üretmek senin uzmanlık alanın.
    Olabildiğince kısa ve anlaşılır cümleler kuruyorsun.
    Eğer kullanacağın bilgilerde eksikler varsa, araştırmacıya soruyorsun.
    """,
    verbose=True,
    allow_delegation=True   # Eksik bilgileri araştırmacıya sorabilmek için
)

# Görevleri tanımlayın
research_task = Task(
    description=f"""Verilen veri kümesi ({df_merged.head()}) hakkında detaylı analiz yap ve konuyu araştır.""",
    expected_output='Mevsim, uçak tipi , uçuş süresi gibi değişkenlerle fiyatlar arasındaki ilişkinin yorumlanması',
    agent=analyst,
    verbose=True
)

write_task = Task(
    description=f"""Araştırmacının sağladığı bilgileri kullanarak bilgilendirici ve uzman diliyle yazılmış bir açıklama metini yaz. 
    Metin, her değişken arasındaki ilişki için olası optimal fiyat aralıklarını vermeli ve sebeplerini açıklamalıdır.
    """,
    expected_output='En yüksek kar için dönem, uçuş süresi ,uçuş sınıfı gibi parametrelere bakarak optimal fiyat aralıklarını belirlemelidir.',
    agent=writer,
    verbose=True
)

# CrewAI ekibini oluşturun
crew = Crew(
    agents=[analyst, writer],
    tasks=[research_task, write_task],
    verbose=2,
    process=Process.sequential  # Görevleri sıralı olarak çalıştır
)

# Projeyi başlatın
t1 = time.time()
result = crew.kickoff()
print(result)
show_time("Toplam süre ", t1)


## qdrant

In [52]:
from typing import List
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [53]:
# QdrantClient ve SentenceTransformer örnekleri
qdrant = QdrantClient(url="http://eager_cerf:6333")
encoder = SentenceTransformer('all-MiniLM-L6-v2')  # Kullanılacak modelin belirlenmesi

In [56]:
document = ["Optimal Price Ranges for Each Variable and Their Relationships: Based on the analysis, we can conclude that the optimal price ranges for each variable and their relationships are as follows: **Flight Duration:** * Short flights (less than 2 hours): ₹8,000 - ₹12,000 * Medium flights (2-4 hours): ₹12,000 - ₹18,000 * Long flights (more than 4 hours): ₹18,000 - ₹25,000 **Departure Date:** * Peak travel seasons (June, July, and August): ₹15,000 - ₹25,000 * Off-peak travel seasons: ₹10,000 - ₹18,000 **Airline:** * Premium airlines: ₹20,000 - ₹40,000 * Mid-range airlines: ₹15,000 - ₹30,000 * Budget airlines: ₹8,000 - ₹20,000 **Class:** * Economy class: ₹8,000 - ₹18,000 * Business class: ₹20,000 - ₹40,000 * First class: ₹30,000 - ₹60,000 **Interaction between Airline and Class:** * Premium airlines with economy class: ₹15,000 - ₹25,000 * Premium airlines with business class: ₹30,000 - ₹50,000 * Premium airlines with first class: ₹45,000 - ₹70,000 * Budget airlines with economy class: ₹8,000 - ₹15,000 * Budget airlines with business class: ₹15,000 - ₹25,000 * Budget airlines with first class: ₹20,000 - ₹35,000 These optimal price ranges take into account the correlations between flight duration, departure date, airline, and class, as well as the interactions between these variables. By understanding these relationships, airlines can inform their pricing strategies and customer segmentation to maximize revenue and appeal to a wider range of customers."] 

In [54]:
# Koleksiyon oluşturma ve vektör parametreleri tanımlama
qdrant.create_collection(
        collection_name="flights",
        vectors_config=models.VectorParams(
            size=encoder.get_sentence_embedding_dimension(),
            distance=models.Distance.COSINE
        )
)

True

In [57]:
# Puanları yükleme işlemi
text = document[0]
points = [
    models.PointStruct(
        id=0,
        vector=encoder.encode(text).tolist(),
        payload={"description": text}
    )
]

qdrant.upload_points(
    collection_name="flights",
    points=points,
    batch_size=64,
    parallel=1,
    method=None,
    max_retries=3,
    wait=False
)

In [28]:
import json
output_file = "encoded_documents.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump([point.payload for point in points], f, ensure_ascii=False, indent=2)

print(f"Documents saved to {output_file}")

Documents saved to encoded_documents.json


In [58]:
# Semantik arama işlemleri
query_vector = encoder.encode("Flight Duration").tolist()
hits = qdrant.search(
    collection_name="flights",
    query_vector=query_vector,
    limit=3
)
for hit in hits:
    print(hit.payload, "score:", hit.score)


{'description': 'Optimal Price Ranges for Each Variable and Their Relationships: Based on the analysis, we can conclude that the optimal price ranges for each variable and their relationships are as follows: **Flight Duration:** * Short flights (less than 2 hours): ₹8,000 - ₹12,000 * Medium flights (2-4 hours): ₹12,000 - ₹18,000 * Long flights (more than 4 hours): ₹18,000 - ₹25,000 **Departure Date:** * Peak travel seasons (June, July, and August): ₹15,000 - ₹25,000 * Off-peak travel seasons: ₹10,000 - ₹18,000 **Airline:** * Premium airlines: ₹20,000 - ₹40,000 * Mid-range airlines: ₹15,000 - ₹30,000 * Budget airlines: ₹8,000 - ₹20,000 **Class:** * Economy class: ₹8,000 - ₹18,000 * Business class: ₹20,000 - ₹40,000 * First class: ₹30,000 - ₹60,000 **Interaction between Airline and Class:** * Premium airlines with economy class: ₹15,000 - ₹25,000 * Premium airlines with business class: ₹30,000 - ₹50,000 * Premium airlines with first class: ₹45,000 - ₹70,000 * Budget airlines with economy

In [51]:
qdrant.delete_collection(collection_name="flights")

True

In [60]:
[hit.payload.get('description', '') for hit in hits]

['Optimal Price Ranges for Each Variable and Their Relationships: Based on the analysis, we can conclude that the optimal price ranges for each variable and their relationships are as follows: **Flight Duration:** * Short flights (less than 2 hours): ₹8,000 - ₹12,000 * Medium flights (2-4 hours): ₹12,000 - ₹18,000 * Long flights (more than 4 hours): ₹18,000 - ₹25,000 **Departure Date:** * Peak travel seasons (June, July, and August): ₹15,000 - ₹25,000 * Off-peak travel seasons: ₹10,000 - ₹18,000 **Airline:** * Premium airlines: ₹20,000 - ₹40,000 * Mid-range airlines: ₹15,000 - ₹30,000 * Budget airlines: ₹8,000 - ₹20,000 **Class:** * Economy class: ₹8,000 - ₹18,000 * Business class: ₹20,000 - ₹40,000 * First class: ₹30,000 - ₹60,000 **Interaction between Airline and Class:** * Premium airlines with economy class: ₹15,000 - ₹25,000 * Premium airlines with business class: ₹30,000 - ₹50,000 * Premium airlines with first class: ₹45,000 - ₹70,000 * Budget airlines with economy class: ₹8,000 

In [79]:
import os
import time
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool

def show_time(message, t1):
    t2 = time.time()
    print(f"{message}: {round(t2-t1,4)} saniye")
    return t2

# OpenAI ortam değişkenlerini tanımlayın
os.environ["OPENAI_API_BASE"] = 'https://api.groq.com/openai/v1'
os.environ["OPENAI_MODEL_NAME"] = 'llama3-70b-8192'
os.environ["OPENAI_API_KEY"] = "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"


# Ajanları tanımlayın
analyst = Agent(
    role='Veri Analisti',
    goal='Verilen veri kümesi hakkında detaylı analiz yap ve veri ile ilgili bilgileri araştır.',
    backstory="""
    Sen uzman bir araştırmacısın. 
    Araştırma yaparken İngilizce kullanıyorsun.
    Teknik konulara hakimsin. 
    Belirli bir veri kümesi hakkında derinlemesine bilgi toplaman gerekiyor.
    """,
    verbose=True,
    allow_delegation=False
)

writer = Agent(
    role='Raporlayıcı',
    goal='Araştırmacının sağladığı bilgileri kullanarak en yüksek geliri elde edebilmek için uçak fiyatları nasıl belirlenmelidir hangi parametreler daha etkilidir yorumlamalar yap.',
    backstory="""
    Sen uzman bir fiyat raporlayıcısısın. 
    Teknik konulara hakimsin. 
    Teknik ve bilgilendirici içerik üretmek senin uzmanlık alanın.
    Olabildiğince kısa ve anlaşılır cümleler kuruyorsun.
    Eğer kullanacağın bilgilerde eksikler varsa, araştırmacıya soruyorsun.
    """,
    verbose=True,
    allow_delegation=True   # Eksik bilgileri araştırmacıya sorabilmek için
)

# Görevleri tanımlayın
research_task = Task(
    description=f"""Verilen veri kümesi ({df_merged.head()}) hakkında detaylı analiz yap ve konuyu araştır.""",
    expected_output='Mevsim, uçak tipi , uçuş süresi gibi değişkenlerle fiyatlar arasındaki ilişkinin yorumlanması',
    agent=analyst,
    verbose=True
)

write_task = Task(
    description=f"""Araştırmacının sağladığı bilgileri kullanarak bilgilendirici ve uzman diliyle yazılmış bir açıklama metini yaz. 
    Metin, her değişken arasındaki ilişki için olası optimal fiyat aralıklarını vermeli ve sebeplerini açıklamalıdır.
    """,
    expected_output='Optimal Price Ranges for Each Variable and Their Relationships: Based on the analysis, we can conclude that the optimal price ranges for each variable and their relationships are as follows: **Flight Duration:** * Short flights * Medium flights  * Long flights **Departure Date:** * Peak travel seasons (June, July, and August) * Off-peak travel seasons  **Class:** * Economy class * Business class',
    agent=writer,
    verbose=True
)

# CrewAI ekibini oluşturun
crew = Crew(
    agents=[analyst, writer],
    tasks=[research_task, write_task],
    verbose=2,
    process=Process.sequential  # Görevleri sıralı olarak çalıştır
)

# Projeyi başlatın
t1 = time.time()
result = crew.kickoff()
print(result)
show_time("Toplam süre ", t1)


2024-07-18 05:35:42,247 - 281472894916864 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-07-18 05:35:42][DEBUG]: == Working Agent: Veri Analisti
 [2024-07-18 05:35:42][INFO]: == Starting Task: Verilen veri kümesi (     id   flight source_city destination_city  price      departure_date  \
0   180  6E-2193       Delhi           Mumbai  11814 2024-07-17 08:19:44   
1   542   UK-833       Delhi           Mumbai  14565 2024-07-16 14:27:58   
2   693   AI-506       Delhi           Mumbai  12150 2024-07-16 11:09:00   
3   778   AI-665       Delhi           Mumbai   5955 2024-07-16 15:42:34   
4  1242   AI-479       Delhi           Mumbai  12740 2024-07-16 15:46:10   

         arrival_date    class    airline  
0 2024-07-17 12:09:32  Economy     Indigo  
1 2024-07-17 05:53:10  Economy    Vistara  
2 2024-07-16 22:43:48  Economy  Air_India  
3 2024-07-16 17:52:46  Economy  Air_India  
4 2024-07-17 09:31:10  Economy  Air_India  ) hakkında detaylı analiz yap ve konuyu araştır.


> Entering new CrewAgentExecutor chain...
Thought: I have carefully examined the given dataset an

1721281054.1746633

In [ ]:
"""Optimal Price Ranges for Each Variable and Their Relationships:

Based on the analysis, we can conclude that the optimal price ranges for each variable and their relationships are as follows:

**Flight Duration:**

* Short flights (less than 2 hours): ₹8,000 - ₹12,000
* Medium flights (2-4 hours): ₹12,000 - ₹18,000
* Long flights (more than 4 hours): ₹18,000 - ₹25,000

**Departure Date:**

* Peak travel seasons (June, July, and August): ₹15,000 - ₹25,000
* Off-peak travel seasons: ₹10,000 - ₹18,000

**Airline:**

* Premium airlines: ₹20,000 - ₹40,000
* Mid-range airlines: ₹15,000 - ₹30,000
* Budget airlines: ₹8,000 - ₹20,000

**Class:**

* Economy class: ₹8,000 - ₹18,000
* Business class: ₹20,000 - ₹40,000
* First class: ₹30,000 - ₹60,000

**Interaction between Airline and Class:**

* Premium airlines with economy class: ₹15,000 - ₹25,000
* Premium airlines with business class: ₹30,000 - ₹50,000
* Premium airlines with first class: ₹45,000 - ₹70,000
* Budget airlines with economy class: ₹8,000 - ₹15,000
* Budget airlines with business class: ₹15,000 - ₹25,000
* Budget airlines with first class: ₹20,000 - ₹35,000

These optimal price ranges take into account the correlations between flight duration, departure date, airline, and class, as well as the interactions between these variables. By understanding these relationships, airlines can inform their pricing strategies and customer segmentation to maximize revenue and appeal to a wider range of customers."""

In [3]:
from sqlalchemy import create_engine, MetaData, Table

# PostgreSQL bağlantı bilgileri
username = 'postgres'
password = '1009'
host = 'postgres'
port = '5432'
database_name = 'postgres'

# PostgreSQL bağlantı URL'si oluşturma
conn_url = f'postgresql://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(conn_url)

# SQLAlchemy MetaData objesi oluşturma
metadata = MetaData()

# Silinecek tablonun adı
table_name = 'flight'

# Var olan tabloyu yükleme (gerekirse)
metadata.reflect(bind=engine)

# Tabloyu MetaData'dan al
table = metadata.tables.get(table_name)

# Eğer tablo varsa sil
if table is not None:
    table.drop(engine)
    print(f"'{table_name}' adındaki tablo başarıyla silindi.")
else:
    print(f"'{table_name}' adında bir tablo bulunamadı.")


'flight' adındaki tablo başarıyla silindi.


In [106]:
conn = psycopg2.connect(**config["POSTGRES"])

OperationalError: connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


In [103]:
from datetime import datetime, timedelta
from qdrant_client import models
from sentence_transformers import SentenceTransformer
 

config = {
            "POSTGRES": {
                'user': '',
                'password': '',
                'host': '',
                'port': '',
                'dbname': ''
            },

            "DATES": {
                        'end date': datetime.now(),
                        'start date': datetime.now() - timedelta(days=1)
                    },

            "COLUMNS": {"['id', 'flight', 'source_city', 'destination_city','price','departure_date','arrival_date','class','airline']"} ,

            "OPENAI" : {
                        "OPENAI_API_BASE" : 'https://api.groq.com/openai/v1',
                        "OPENAI_MODEL_NAME" : 'llama3-70b-8192' ,
                        "OPENAI_API_KEY" : b'gAAAAABmoojveq_MTKTbTTOgVUkEZFDhqCwhIjLAivpR35ziWazztD8tlhjH-RHvBdkCMiCl0kaifRrQB069rdnXFbjGGmVNliYj1zFfXcIDTVlUVcG7sUkPbn19QtKtksAg0zALXQEcEHo1ZmiRveT2WmqYgGEnuQ=='
                        },

            "QDRANT CLIENT" : {""},

            "ENCODER" : {SentenceTransformer('all-MiniLM-L6-v2')},

        }



In [105]:

from qdrant_client import QdrantClient
import os

import os
from qdrant_client import QdrantClient



config.update({
                "POSTGRES": {
                    'user': 'postgres',
                    'password': 'gAAAAABmooh8623Co-zymvdAh1YOjvQONpHi6fisdQ-T5f28ls0LN56QYseqtz81VqATGzB8HQpRKDnUl-PYtfdhgPdjl9pxrQ==',
                    'host': 'localhost',
                    'port': '5432',
                    'dbname': 'postgres'
                },
                
                "QDRANT CLIENT": QdrantClient(url="http://eager_cerf:6333")
            })



# conn_url = f'postgresql://{"username"}:{"password"}@{"host"}:{"port"}/{"database_name"}'
conn_url = f'postgresql://{config["POSTGRES"]["user"]}:{config["POSTGRES"]["password"]}@{config["POSTGRES"]["host"]}:{config["POSTGRES"]["port"]}/{config["POSTGRES"]["dbname"]}'




In [110]:
config["POSTGRES"]

{'user': 'postgres',
 'password': 'gAAAAABmooh8623Co-zymvdAh1YOjvQONpHi6fisdQ-T5f28ls0LN56QYseqtz81VqATGzB8HQpRKDnUl-PYtfdhgPdjl9pxrQ==',
 'host': 'localhost',
 'port': '5432',
 'dbname': 'postgres'}

In [113]:
from cryptography.fernet import Fernet

class Decrypt():

    def decrypt_message(self,encrypted_message, key):

        fernet = Fernet(key)
        decrypted = fernet.decrypt(encrypted_message).decode()
    
        return decrypted

In [114]:
class Parser(Decrypt):
    def parser_config(self):
        parser = argparse.ArgumentParser()

        # Argümanları ekle
        parser.add_argument('--env', type=str, required=True, help='Environment')
        parser.add_argument('--key', type=str, required=True, help='Encrypted key')

        # Argümanları parse et
        args = parser.parse_args()

        # Argümanları sözlük olarak al
        # args_dict = vars(args)
        # key = args_dict['key']
        
        # #env = args_dict['env']
       
        # Şifreli anahtar kullanarak config değerlerini güncelle
        config["POSTGRES"]["password"] = self.decrypt_message(config["POSTGRES"]["password"], b'ytrhgFTEDMW90vcB3lpyvCFHhoAMP1V8SV2JZUiMwQI=')
        config["OPENAI"]["OPENAI_API_KEY"] = self.decrypt_message(config["OPENAI"]["OPENAI_API_KEY"], b'ytrhgFTEDMW90vcB3lpyvCFHhoAMP1V8SV2JZUiMwQI=')

        return config

In [116]:
import argparse

In [117]:
c = Parser()
c.parser_config()

usage: ipykernel_launcher.py [-h] --env ENV --key KEY
ipykernel_launcher.py: error: the following arguments are required: --env, --key


SystemExit: 2

In [119]:
config["POSTGRES"]["password"] = c.decrypt_message(config["POSTGRES"]["password"], b'ytrhgFTEDMW90vcB3lpyvCFHhoAMP1V8SV2JZUiMwQI=')
config["OPENAI"]["OPENAI_API_KEY"] = c.decrypt_message(config["OPENAI"]["OPENAI_API_KEY"], b'ytrhgFTEDMW90vcB3lpyvCFHhoAMP1V8SV2JZUiMwQI=')


In [120]:
config["POSTGRES"]["password"]

'1009'

In [121]:
config["OPENAI"]["OPENAI_API_KEY"] 

'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'